Necessary imports

In [24]:
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
import numpy as np

if os.environ['COMPUTERNAME'] == 'FYNN':            # name of surface PC
    sys.path.append(r'C:\Users\Surface\Masterarbeit')
elif os.environ['COMPUTERNAME'] == 'FYNNS-PC':  # desktop name
    sys.path.append(r'C:\Users\test\Masterarbeit')
    
else:
    raise ValueError("Unbekannter Computername: " + os.environ['COMPUTERNAME'])


Load, Transform and Split the data

In [25]:
import data_prep

X_train, X_val, X_test, y_train, y_val, y_test, feature_names = data_prep.load_tranform_and_split_data('C1_V01_delta_kan', split_ratio=(0.6, 0.2, 0.2))

# convert the data to PyTorch tensors
# and add an extra dimension for the target variable
X_train_tensor = torch.from_numpy(X_train).float()
X_val_tensor = torch.from_numpy(X_val).float()
X_test_tensor = torch.from_numpy(X_test).float() 
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1) # Add extra dimension for compatibility
y_val_tensor = torch.from_numpy(y_val).float().unsqueeze(1)
y_test_tensor = torch.from_numpy(y_test).float().unsqueeze(1)
print(X_train_tensor.shape, X_val_tensor.shape, X_test_tensor.shape)
print(feature_names)

torch.Size([31206, 37]) torch.Size([10402, 37]) torch.Size([10403, 37])
Index(['Ist_Blechhaltergewicht', 'Ist_Gegenhaltekraft_HL_1', 'T2', 'K1', 'T20',
       'K7', 'T23', 'K8', 'DS_10', 'GS_10', 'SD_10', 'LS_10', 'PP_10', 'TT_10',
       'TM5_10', 'RF_10', 'TD_10', 'is_weekend', 'dayofweek_sin',
       'dayofweek_cos', 'month_sin', 'month_cos', 'hour_sin', 'hour_cos',
       'day_sin', 'day_cos', 'quarter_sin', 'quarter_cos', 'week_sin',
       'week_cos', 'Diff_Hubzahl', 'Diff_Ziehtiefe',
       'Diff_Ziehkissenverstellu', 'Diff_Stoesselverstellung-mm',
       'Diff_Gewichtsausgleich', 'BT_NR_freq', 'STP_NR_freq'],
      dtype='object')


Create an Ensemble

In [26]:
import NN_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# number of networks
n = 5
nets_ops = []
for i in range(n):
    net = NN_model.Custom_NN_Model(input_dim=X_train.shape[1], hidden_dims=[256, 64, 16], output_dim=1, do_rate=0, loss_type='heteroscedastic').to(device)  # Create model instance and move to device
    optimizer = torch.optim.Adam(net.parameters(), lr=0.0042, weight_decay=0.0001)  # Create optimizer
    nets_ops.append((net, optimizer))
    
print(nets_ops[0])  # Print the first model to check if it was created correctly

Using device: cpu
(Custom_NN_Model(
  (hidden_layers): Sequential(
    (0): Linear(in_features=37, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0, inplace=False)
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0, inplace=False)
    (6): Linear(in_features=64, out_features=16, bias=True)
    (7): ReLU()
    (8): Dropout(p=0, inplace=False)
  )
  (mean_layer): Linear(in_features=16, out_features=1, bias=True)
  (var_layer): Linear(in_features=16, out_features=1, bias=True)
), Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0042
    maximize: False
    weight_decay: 0.0001
))


In [ ]:
model = NN_model.Custom_NN_Model(input_dim=X_train.shape[1], hidden_dims=[256, 64, 16], output_dim=1, do_rate=0, loss_type='heteroscedastic').to(device)  # Create model instance and move to device

prediction = model(X_train_tensor[0])
mu, log_var = prediction = model(X_train_tensor[0])
print(type(prediction))
print(f"Prediction for first training sample: {prediction[0].item()}")
print(f"Prediction for first training sample: {prediction[1].item()}")
print(mu)



<class 'tuple'>
Prediction for first training sample: -0.012562423944473267
Prediction for first training sample: 0.0683891698718071


Method for Training the Ensembles

In [ ]:
def train_model(net, optimizer, batch_size=64, n_epochs=1000, patience=20, device = 'cpu'):
    
    # DataLoader for batching the data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
        
    # Early Stopping values
    best_val_loss = np.inf
    epochs_no_improve = 0
    loss_history = []
    val_loss_history = []
    net.to(device)  # Move model to device
    for epoch in range(n_epochs):
        net.train()
        batch_losses = []
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            loss = NN_model.heteroscedastic_loss(net, X_batch, y_batch) # does a forward pass and computes the loss  
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())
        loss_history.append(loss.item())
            
        # calculate validation loss
        model.eval()                            # Set model to evaluation mode
        with torch.no_grad():

            # Forward pass
            val_loss = NN_model.heteroscedastic_loss(model, X_val_tensor, y_val_tensor)
                
            val_loss_history.append(val_loss.item())
            
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
            print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {np.mean(batch_losses):.4f}, Val Loss: {val_loss.item():.4f}, Best Val Loss: {best_val_loss:.4f}")
            
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}, Best Val Loss: {best_val_loss:.4f}")
                model.load_state_dict(best_model_state)
                break    
            
    
    outputs = [net(X_test_tensor)]
    outputs = np.hstack(torch.stack(outputs).detach().numpy())
    
    return loss_history, val_loss_history, outputs

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
outputs = []
loss_histories = []
val_loss_histories = []
for net, ops in nets_ops:
    loss_history, val_loss_history, outputs = train_model(net, ops, device=device)
    outputs.append(outputs)
    loss_histories.append(loss_history)
    val_loss_histories.append(val_loss_history)
    
outputs = np.vstack(outputs)

Epoch 1/1000, Train Loss: 57.6877, Val Loss: 6416.5942, Best Val Loss: 6416.5942


KeyboardInterrupt: 

In [ ]:
#! handling the test data
# Select a random subset of test data for visualization
num_points = 100
np.random.seed(42)

random_indices = np.random.choice(len(X_test), num_points, replace=False)
random_indices.sort()

# Calculate the mean and standard deviation of the predictions on the test data
mean_test_pred_heteroscedastic = np.mean(outputs, axis = 0)

# # Calculate epistemic
# epistemic_var_heteroscedastic = np.var(y_test_pred_heteroscedastic_mean, axis=0)
# print(f"Epistemic Variance: {epistemic_var_heteroscedastic.mean():.6f}")

# # Calculate aleatoric variance (heteroscedastic)
# aleatoric_var_heteroscedastic = np.mean(y_test_pred_heteroscedastic_var, axis=0)
# print(f"Aleatoric Variance: {aleatoric_var_heteroscedastic.mean():.6f}")

# # Calculate total standard deviation
# total_std_heteroscedastic = np.sqrt(epistemic_var_heteroscedastic + aleatoric_var_heteroscedastic)
# print(f"Total Standard Deviation: {total_std_heteroscedastic.mean():.6f}")

#print(f' y_test std: {y_test_tensor.std().item()}')

# Calculate R² score for the test data
r2_test = r2_score(y_test_tensor.cpu(), mean_test_pred_heteroscedastic)
print(f"R² on Test Data: {r2_test:.3f}")

# Assign descriptive variable names for MC Dropout mean and standard deviation
mc_mean = mean_test_pred_heteroscedastic  # Predicted mean for each test point
mc_std = np.std(outputs, axis = 0)   # Predicted standard deviation for uncertainty

# Define the confidence interval bounds (95% CI ≈ mean ± 2*std)
mc_lower_bound = (mc_mean - 2 * mc_std).reshape(-1)  # Ensure it is a 1D array
mc_upper_bound = (mc_mean+ 2 * mc_std).reshape(-1)  # Ensure it is a 1D array

print(f"Shape of mc_lower_bound: {mc_lower_bound.shape}, mc_upper_bound: {mc_upper_bound.shape}")

# calculate the coverage of the confidence interval
in_interval = (y_test >= mc_lower_bound) & (y_test <= mc_upper_bound)
# number of true
counter = 0
# Print whether each true value is within the confidence interval
for i in in_interval:
    if i:
        counter += 1

print(f"Number of true values within the confidence interval: {counter} out of {len(in_interval)}")

# Calculate coverage (percentage of true values within the CI)
coverage = np.mean(in_interval) * 100  # in percentage

# Print the coverage value
print(f"Coverage: {coverage:.2f}%")

# Check whether each true value from random indices lies within the 95% confidence interval
# If yes, the point will be green; if not, red
in_interval_rand_indi = (y_test[random_indices] >= mc_lower_bound[random_indices]) & (y_test[random_indices] <= mc_upper_bound[random_indices])
colors = ['tab:green' if inside else 'tab:red' for inside in in_interval_rand_indi]

# Create the plot
plt.figure(figsize=(12, 5))
x_axis = np.arange(num_points)  # Create an index axis for plotting

# Plot the predicted mean
plt.plot(x_axis, mc_mean[random_indices], label="Prediction (mean)", color='tab:blue')

# Plot the confidence interval as a shaded region
plt.fill_between(x_axis, mc_lower_bound[random_indices], mc_upper_bound[random_indices], alpha=0.4,
                 color='tab:blue', label='95% Confidence Interval')

# Scatter plot of true values with color-coded points based on interval inclusion
plt.scatter(x_axis, y_test_tensor[random_indices].cpu().flatten(), label="True Values", c=colors, s=25, zorder=3)

# Final plot settings
plt.title("MC Dropout Prediction with Uncertainty")
plt.xlabel("Test Point Index")
plt.ylabel("x_Einzug [mm]")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()